In [1]:
from __future__ import print_function



In [2]:
"""DVS128 closed loop oscillator routine by Alexander Rivkind, Eldad Assa, Michael Kreiserman and Ehud Ahissar
based on: 
*DVS example by: Yuhuang Hu (duguyue100@gmail.com)
*"Syclop" paradigm: Ahissar and Assa 2017
"""
from reservoir import ESN
import numpy as np
import time
import SYCLOP_env as syc
import matplotlib.pyplot as plt


import pickle
from RL_brain_b import DeepQNetwork

import cv2
from misc import *
import maestro
import teachers
from pyaer.dvs128 import DVS128
%matplotlib notebook

In [3]:
class Free_run_agent():
    def __init__(self,max_q = None):
        self.hp = HP()
        self.hp.action_space = ['v_right','v_left','v_up','v_down','null'] + \
                               [['v_right','v_up'],['v_right','v_down'],['v_left','v_up'],['v_left','v_down']]#'
        self.hp.max_speed  = 10
        self.q_centre = np.array([0.,0.], dtype='f') 
        self.reset()

    def reset(self):
        self.q_ana = np.array([0.,0.], dtype='f') 
        self.qdot = np.array([0.0,0.0])
        self.qdotdot = np.array([0.0,0.0])
        self.q = np.int32(np.floor(self.q_ana))


    def act(self,a):
        if a is None:
            action = 'null'
        else:
            action = self.hp.action_space[a]
        #delta_a = 0.001
        if type(action) == list:
            for subaction in action:
                self.parse_action(subaction)
        else:
            self.parse_action(action)

        #print('debug', self.max_q, self.q_centre)
        self.qdot += self.qdotdot
        #self.qdot -= self.hp.returning_force*(self.q_ana-self.q_centre)
        self.q_ana +=self.qdot
        self.q = np.int32(np.floor(self.q_ana))

    def parse_action(self,action):
        if type(action)==int:
            self.qdot[0] = action
            self.qdotdot = np.array([0., 0.])
        elif action == 'v_up':   # up
            self.qdot[1] = self.qdot[1] + (1 if self.qdot[1] < self.hp.max_speed else 0)
            self.qdotdot = np.array([0., 0.])
        elif action == 'v_down':   # down
            self.qdot[1] = self.qdot[1] - (1 if self.qdot[1] > -self.hp.max_speed else 0)  
            self.qdotdot = np.array([0., 0.])
        elif action == 'v_left':   # left
            self.qdot[0] = self.qdot[0] - (1 if self.qdot[0] > -self.hp.max_speed else 0)  
            self.qdotdot = np.array([0., 0.])
        elif action == 'v_right':   # right
            self.qdot[0] = self.qdot[0] + (1 if self.qdot[0] < self.hp.max_speed else 0) 
            self.qdotdot = np.array([0.,0.])
        elif action == 'null':   # null
            pass
        else:
            error('unknown action')

In [4]:
import pickle

In [5]:
    hp = HP()
    hp.mem_depth=1
    hp.logmode = False
#     recorder = Recorder(n=4)

    import pickle

    sensor = syc.Sensor()
    agent = Free_run_agent()

    reward = syc.Rewards()
    observation_size = 256*4
    RL = DeepQNetwork(len(agent.hp.action_space), observation_size*hp.mem_depth,#sensor.frame_size+2,
                      reward_decay=0.99,
                      e_greedy=0.99,
                      e_greedy0=0.99,
                      replace_target_iter=10,
                      memory_size=100000,
                      e_greedy_increment=0.0001,
                      learning_rate=0.0025,
                      double_q=False,
                      dqn_mode=True,
                      state_table=np.zeros([1,observation_size*hp.mem_depth])
                      )
    RL.dqn.load_nwk_param('saved_runs/run_syclop_lirondb.py_noname_1557658952/best_liron.nwk')


In [6]:
servo = maestro.Controller(ttyStr='/dev/ttyACM0')

In [7]:
def gohome():    
    servo.setSpeed(0,0)
    servo.setSpeed(1,0)
    servo.setTarget(1,7500)     #set speed of servo 1
    servo.setTarget(0,5000)  #set servo to move to center position
    servo.setSpeed(0,1)
    servo.setSpeed(1,1)
    time.sleep(1.0)

In [8]:
gohome()

In [9]:
servo.moveByVelocity(0,-1)

In [10]:
def local_observer(sensor,agent):
    if hp.logmode:
        normfactor=1.0
    else:
        normfactor = 1.0/1.0
    return normfactor*np.concatenate([relu_up_and_down(sensor.central_dvs_view),
            relu_up_and_down(cv2.resize(1.0*sensor.dvs_view, dsize=(16, 16), interpolation=cv2.INTER_AREA))])

observation = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])
hp.fading_mem = 0.5

In [11]:
training = True
device = DVS128()


device.start_data_stream()
# load new config
# device.set_bias_from_json("./scripts/configs/dvs128_config.json")
# print (device.get_bias())

clip_value = 3
histrange = [(0, v) for v in (128, 128)]
t_prev = time.time()
dt_nom = 0.025
# dt_list = []
cnt = 0
empty_cnt = 0
recorder = Recorder(n=4)
step = 0 
observation = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])
observation_ = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])

agent.reset()

while True:
    t_this = time.time()
    try:
        (pol_events, num_pol_event,
         special_events, num_special_event) = \
            device.get_event("events_hist")
        if t_this - t_prev > dt_nom:
                t_prev=t_this
                if num_pol_event != 0:
                    img = pol_events[..., 1]-pol_events[..., 0]

                    action = RL.choose_action(observation.reshape([-1]))
#                     reward.update_rewards(sensor = sensor, agent = agent)
#                     recorder.record([agent.q_ana[0],agent.q_ana[1],reward.reward,RL.epsilon])
                    agent.act(action)
                    for orientation in [0,1]:
                        servo.moveByVelocity(orientation,(int(np.round(agent.qdot[orientation]))))
                    sensor.dvs_view = cv2.resize(1.0*img, dsize=(64, 64))
                    sensor.central_dvs_view = cv2.resize(1.0*img[64-8:64:8,64-8:64+8], dsize=(16, 16))
                    print(np.min(sensor.dvs_view ),np.max(sensor.dvs_view ) )
                    observation *= hp.fading_mem
                    observation += local_observer(sensor, agent)  # todo: generalize
                    if step%1000 ==0:
                        print('step',step)
                    step += 1
    except KeyboardInterrupt:
            device.shutdown()
            break
            

-0.75 0.75
step 0
-0.75 1.0
-1.25 1.75
-2.5 2.5
-3.25 3.0
-2.75 3.75
-3.25 3.5
-2.5 3.75
-3.75 4.0
-3.5 4.75
-3.5 6.0
-2.75 4.25
-3.5 5.75
-4.0 6.0
-3.5 6.0
-3.5 5.5
-3.5 5.5
-5.0 6.25
-3.75 5.25
-2.0 2.0
-1.25 0.5
-0.25 0.5
-0.25 0.25
-0.25 0.25
-1.25 0.25
-0.25 0.25
-2.5 2.0
-2.75 2.5
-2.5 2.75
-3.0 2.75
-3.0 2.25
-4.0 3.25
-4.0 5.25
-5.0 5.5
-5.5 5.5
-5.0 5.5
-5.25 6.0
-5.25 5.75
-6.25 5.75
-4.5 5.25
-4.5 4.25
-4.5 5.0
-3.5 4.25
-4.0 5.5
-4.25 5.0
-3.75 5.0
-4.5 6.25
-4.5 5.5
-3.75 5.5
-4.5 5.5
-3.25 5.0
-3.5 5.5
-3.75 5.75
-4.0 5.25
-4.0 5.5
-3.75 5.25
-4.0 5.5
-4.0 5.5
-4.25 5.25
-3.25 5.25
-4.0 4.5
-3.5 4.5
-4.0 3.75
-2.0 2.0
-4.25 2.5
-3.0 2.0
-3.75 3.0
-4.0 4.25
-3.5 3.25
-4.75 3.0
-3.25 2.25
-3.0 2.0
-2.25 2.0
-4.5 2.5
-5.0 2.75
-4.25 3.25
-4.0 4.25
-3.5 4.25
-3.5 4.75
-2.25 2.25
-2.0 2.5
-0.75 0.5
-1.5 0.25
-1.75 1.25
-1.5 1.25
-1.5 1.0
-1.25 0.75
-2.0 1.25
-2.5 1.75
-3.5 2.0
-2.0 1.5
-3.0 3.75
-3.5 3.5
-2.75 2.75
-2.5 3.75
-4.0 5.5
-5.0 5.25
-3.5 5.75
-3.5 5.25
-5.75 5.5
-4.

-7.75 0.5
-7.0 0.5
-7.25 0.5
-6.25 0.5
-7.5 0.5
-5.25 0.5
-5.75 0.5
-5.5 0.5
-5.0 0.75
-4.25 4.5
-4.5 3.75
-3.75 3.25
-1.0 1.75
-3.25 4.25
-4.5 3.5
-1.0 4.25
-2.25 5.0
-1.5 5.25
-2.0 4.75
-1.75 5.0
-1.25 4.75
-1.75 4.5
-3.5 4.5
-4.25 5.25
-4.5 4.75
-3.25 5.25
-3.25 5.0
-3.0 4.5
-2.75 5.0
-3.5 6.5
-3.25 7.0
-3.75 6.5
-4.0 6.25
-3.75 6.25
-4.0 5.5
-4.0 5.75
-4.5 6.5
-4.5 5.5
-4.75 6.0
-5.0 6.0
-5.25 5.75
-4.75 5.75
-5.25 4.0
-4.25 4.75
-1.5 0.75
-1.25 0.75
-2.0 2.5
-4.5 3.0
-2.75 2.5
-2.75 4.5
-3.5 4.25
-3.75 5.0
-3.25 4.75
-4.25 5.75
-4.5 6.0
-4.75 4.75
-4.0 5.5
-4.5 5.5
-4.0 5.5
-4.0 5.5
-4.5 5.75
-4.0 5.5
-3.5 5.0
-3.75 4.25
-4.25 5.25
-4.5 5.0
-3.75 4.75
-3.75 4.25
-3.0 3.5
-3.25 3.0
-3.25 3.0
-3.25 3.75
-3.25 4.0
-2.75 3.25
-3.25 3.25
-1.75 1.75
-1.75 1.25
-1.75 0.75
-1.25 2.25
-2.0 4.75
-3.0 5.75
-3.5 6.0
-4.0 7.0
-4.25 6.5
-1.0 1.0
-1.0 1.0
-1.75 2.0
-1.75 2.0
-2.0 2.0
-2.0 2.0
-2.0 2.0
-2.5 2.5
-3.75 5.75
-4.25 6.0
-4.0 5.75
-4.25 5.25
-3.25 5.0
-3.5 5.0
-3.0 5.0
-2.5 4.25
-2.25 

-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-0.5 0.25
-0.5 0.25
-0.25 0.25
0.0 0.25
-0.5 0.25
-0.25 0.25
-1.0 0.25
-0.25 0.25
0.0 0.25
-0.5 0.25
-0.25 0.25
-1.25 0.25
-0.25 0.5
-0.25 0.25
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-1.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.75 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-1.5 0.25
-0.25 0.25
-0.75 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-1.0 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-1.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-

0.0 0.25
-0.25 0.25
-0.25 0.25
-0.75 0.5
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-0.5 0.5
-1.0 0.5
-1.5 0.75
-2.0 1.0
-2.5 1.0
-0.25 0.5
-3.0 1.5
-1.5 0.5
-3.25 1.25
-0.25 0.25
-1.75 0.75
-0.25 0.25
-0.25 0.25
-0.5 0.25
-0.5 0.25
-1.75 0.5
-0.25 0.25
-0.25 0.25
-1.0 0.25
-0.25 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
0.0 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.0
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.5 0.5
-0.25 0.5
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.75 0.25
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
0.0 0.25
-1.25 0.75
-0.5 0.5
0.0 0.25
-0.25 0.5
0.0 0.25
0.0 0.5
0.0 0.25
-1.25 0.5
-0.5 0.5
-0.25 0.25
-0.25 0.25
-1.5 0.5
-0.25 0.25
-2.25 0.75
-1.25 0.5
-0.25 0.5
-0.25 0.25
-1.0 0.5
-0.25 0.5
-0.25 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-1.0 0.5
0.0 0.25
-

-4.75 3.5
-4.25 4.5
-5.0 6.0
-5.0 5.75
-4.0 6.0
-4.75 6.0
-5.0 6.25
-4.5 5.75
-5.0 4.75
-4.75 4.75
-4.25 4.25
-5.25 3.75
-4.5 4.0
-4.25 4.25
-5.75 4.0
-4.0 3.5
-4.25 3.75
-4.5 3.75
-5.75 4.0
-3.75 3.75
-5.5 4.25
-3.5 3.75
-6.5 5.0
-4.25 6.0
-4.25 6.0
-4.0 5.0
-4.0 5.0
-3.75 4.75
-3.25 4.0
-3.0 3.75
-2.25 2.75
-1.75 0.75
-1.25 0.5
-0.75 0.25
-2.25 1.0
-3.75 2.0
-5.0 2.0
-3.5 1.75
-3.0 1.75
-4.25 2.0
-4.0 3.0
-4.5 4.0
-5.75 4.0
-4.5 4.0
-4.75 4.0
-3.5 3.75
-4.5 4.0
-4.5 4.5
-4.5 4.5
-4.0 3.75
-4.25 4.25
-4.0 3.75
-3.5 5.5
-3.5 5.5
-3.25 4.75
-4.0 6.0
-4.0 5.25
-4.5 5.75
-3.75 6.0
-4.5 6.0
-5.0 5.5
-5.75 7.0
-5.5 6.75
-5.75 6.0
-5.0 5.5
-4.5 5.75
-4.0 5.75
-4.5 5.25
-2.5 1.25
-1.0 0.25
-1.75 1.0
-1.0 0.25
-0.25 0.25
-0.25 0.25
-1.75 0.5
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-1.75 0.25
0.0 0.25
-0.5 0.25
0.0 0.5
0.0 0.25
-1.5 0.25
-2.5 0.5
-1.25 0.5
0.0 0.25
0.0 0.25
0.0 0.25
-1.75 0.5
-2.25 1.5
-2.75 1.25
-2.25 1.25
-2.5 2.0
-4.0 2.0
-4.0 1.5
-3.25 1.25
-1.75 1.5
-2.25 1.5
-3.5 1.75
-2.0 

...

-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.5 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-1.5 0.25
-1.5 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.5 0.25
-0.25 0.25
0.0 0.5
-0.25 0.25
-0.5 0.5
-0.25 0.25
0.0 0.25
0.0 0.25
-0.75 0.5
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-1.0 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.5
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-1.75 0.5
-0.5 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-1.25 0.25
-0.25 0.5
0.0 0.25
-0.25 0.25
-1.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.75 0.25
-0.25 0.25
-0.25 0.25
0.0 0.5
-0.5 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.25
0

0.0 0.25
-1.75 0.75
-1.5 0.5
-1.25 0.5
-0.25 0.25
-0.5 0.25
-0.25 0.25
-1.0 0.5
-2.25 0.75
-1.25 0.5
-1.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.75
-0.25 0.25
-1.25 0.5
-0.25 0.25
-0.5 0.25
-0.75 0.25
-0.75 0.75
-1.0 0.5
-1.0 0.5
-1.0 0.25
-1.25 0.25
0.0 0.25
-0.25 0.25
-1.0 0.5
-0.75 0.75
-1.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.5 0.75
-0.75 0.5
-0.25 0.25
-0.5 0.25
-0.25 0.25
-1.25 0.25
-1.25 0.5
step 13000
-1.5 0.5
-0.5 0.25
-0.75 0.25
-0.25 0.25
-0.5 0.25
-1.5 0.75
-1.75 0.5
-0.5 0.5
-0.5 0.25
-0.25 0.25
-0.25 0.5
-1.0 0.5
-1.25 0.5
-0.5 0.5
-0.25 0.25
-0.25 0.25
-1.0 0.5
-1.25 0.75
-1.25 0.5
-1.5 0.25
-0.25 0.25
-0.25 0.5
-1.0 0.75
-0.25 0.25
-1.25 0.5
-0.5 0.25
-0.25 0.5
-0.75 0.25
-1.5 0.75
-0.25 0.25
-1.5 0.5
-0.75 0.25
-0.25 0.25
0.0 0.25
-1.0 0.5
-1.75 0.5
-1.75 0.5
-0.75 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.5
-1.25 0.5
-0.5 0.25
-0.75 0.5
-0.25 0.25
-0.75 0.75
-1.25 0.75
-1.5 0.75
-1.25 0.5
-0.25 0.25
-0.25 0.25
-1.5 0.75
-1.0 0.5
-0.75 0.5
-0.75 0.25
-

-0.25 0.25
-1.25 0.5
-1.5 0.5
-0.25 0.25
-0.75 0.5
-0.25 0.0
-1.75 0.75
-0.25 0.5
-1.25 0.5
-0.75 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.75 0.75
-1.25 0.5
-1.0 0.25
-1.75 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.0 0.5
-0.25 0.25
-1.25 0.5
-0.75 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-1.5 0.5
-1.5 0.5
-1.75 0.5
-0.5 0.25
-1.0 0.25
-0.5 0.25
-0.25 0.25
-1.25 0.25
-1.5 0.75
-0.75 0.5
-1.25 0.5
-0.25 0.25
-0.5 0.25
0.0 0.25
-0.25 0.25
-1.25 0.5
-0.5 0.5
-1.75 0.5
-1.25 0.5
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.5
-1.75 0.5
-1.0 0.5
-0.75 0.5
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-1.75 0.75
-1.5 0.5
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-2.25 0.5
-1.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.25 0.75
-0.75 0.25
-1.25 0.5
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-1.0 0.5
-1.0 0.5
-1.0 0.25
-0.25 0

-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.75 0.25
-1.75 1.25
-0.25 0.25
-0.25 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.75 0.25
-0.25 0.25
-0.5 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.75 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.5
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
0.0 0.25
0.0 0.25
0.0 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
-0.25 0.25
0.0 0.25
-0.25 0.25
0.0 0.25
0.0 0.25


-0.75 0.5
-0.25 0.25
-0.25 0.25
-0.25 0.5
-1.75 0.5
-1.5 0.5
-1.25 0.75
-0.5 0.25
-1.5 0.5
-1.5 0.25
-1.25 0.5
-1.5 0.5
-0.25 0.25
-0.5 0.5
-1.5 0.25
-1.25 0.5
-1.0 0.5
-1.5 0.75
-0.25 0.25
-2.25 0.5
-0.25 0.25
-1.25 0.5
-0.25 0.5
-0.75 0.25
-0.5 0.25
-1.5 0.25
-1.25 0.5
-0.25 0.5
-1.0 0.5
-1.25 0.5
-1.75 0.25
-1.5 0.5
-0.75 0.5
-1.0 0.5
-0.5 0.25
-1.75 0.75
-1.25 0.25
-0.75 0.25
-0.5 0.25
-0.25 0.25
-2.0 0.5
-1.5 0.5
-1.75 0.75
-0.75 0.25
-1.75 0.75
-0.25 0.25
-0.75 0.25
-1.5 0.5
-0.25 0.25
-0.5 0.25
-0.5 0.25
-0.25 0.25
-1.25 0.5
-1.5 0.25
-0.25 0.25
-1.5 0.5
-0.75 0.5
-1.25 0.5
-0.25 0.25
-1.5 0.25
0.0 0.25
-2.0 0.5
-0.25 0.25
-1.25 0.25
-1.0 0.5
-1.0 0.5
-0.5 0.25


In [17]:
device.shutdown()

In [ ]:
gohome()

In [18]:
60e-3*1e6/3.6e3

16.666666666666668